# Préparation des données
Ce notebook sert à la préparation des données qui seront utilisée plus tard dans le projet. 

Les données proviennent des différents jeux de données disponibles sur GeoAdmin et qui ont été téléchargés en local. Dans chaque section de ce notebook, un lien est disponible pour la récupération des données utilisées, en cas de besoin.    

GeoAdmin propose une API, ainsi que la [documentation](https://api3.geo.admin.ch/index.html) associée, à travers laquelle l'ensemble des données est disponible. Cependant, nous ne pouvont pas l'utiliser dans le cadre de notre projet, les requêtes étant limitées à 20 par minutes, comme stipulé dans les [conditions d'utilisations](https://www.geo.admin.ch/fr/conditions-generales-utilisation-ifdg). En effet, cela prendrait alors beaucoup trop de temps pour enchirir l'ensemble de notre jeu de données.

Nous allons ici nous concentrer sur la Suisse entière. Nous allons créer une grille, afin de séparer la Suisse en un ensemble de zone plus faciles à traiter que les formes géométriques utilisées dans les données de GeoAdmin. Le choix a été fait de prendre des carrés, d'une taille de 1km de côté. Cette valeur pourra être modifiée par la suite, en cas de besoin.

A noter que ces jeux de données utilisent un système de coordonnées [géodésiques](https://fr.wikipedia.org/wiki/G%C3%A9od%C3%A9sique), à savoir le format EPSG:2056 ([LV95](https://fr.wikipedia.org/wiki/Syst%C3%A8me_de_coordonn%C3%A9es_g%C3%A9ographiques_suisse#MN95)).

In [1]:
# Imports nécessaires pour ce notebook
import fiona
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import rasterio
from rasterio.plot import show
import rioxarray
from shapely import Polygon
from shapely.geometry import Point, MultiPolygon, Polygon
import os

## Utilisation des fichiers geodatabase

GeoAdmin propose une partie de ses données sous le format gdb, des geodatabase.
Cette section est là purement à but d'aide à la compréhension sur le fonctionnement de ces fichiers. Les fonctions créées ci-dessous sont réutilisées dans les sections suivantes, afin de permettre de voir ce qui est contenu dans les différents fichiers.

La première étape pour l'utilisation des fichiers gdb est de lister les différentes couches disponibles, afin d'identifier celle qui nous intéresse.

In [2]:
def list_available_layers(gdb_path, to_print = True):
    layers = fiona.listlayers(gdb_path)

    if to_print :
        print(f"Couches disponibles : {layers}")

    return layers

Une fois les couches connues et celles pertinentes identifiées, les données peuvent être extraites des fichiers. En fonction de la taille du fichier, il peut être intéressant de n'avait que les colonnes intéressantes. La fonction ci-dessous permet la récupération du nom des différentes colonnes présentes dans une couche donnée d'un fichier gdb. Ces fichiers contiennent toujours une colonne `geometry`, qui contient un Polygon de la zone.

In [3]:
def list_columns(gdb_path, selected_layer, to_print = True):
    # Lecture de la 1ère ligne du fichier pour la couche souhaitée
    gdf = gpd.read_file(gdb_path, layer=selected_layer, rows=1)

    columns = []
    if not gdf.empty:
        columns = gdf.columns

        if to_print :
            print(f"Colonnes disponibles : {columns.tolist()}")

    return columns

La lecture des fichiers se fait ensuite à l'aide de la fonction `read_file` de `geopandas`, comme cela est fait dans la fonction ci-dessus. Plusieurs paramètres permettent une sélection des attributs ou une zone définie de l'espace à prendre en compte. La documentation de cette fonction est disponible à ce [lien](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html). Le résultat peut ensuite être traité de la même manière que les Dataframe normaux.

## Séparation de la Suisse en cellules
Afin de faciliter la représentation de la distribution du néophyte dans le pays, il a été décidé de travailler avec une grille. Comme expliqué plus haut, le pays a été séparé en un ensemble de carrés, d'une taille de 1km de côté.

La fonction suivante va permettre de faire cela et retournera la grille sous la forme d'une liste de `Polygon`, qui sont nos carrés. De cette amnière, il nous sera plus facile d'utiliser la représentation pixelisée de la Suisse.

In [4]:
# Constantes pour la création de la grille recouvrant la Suisse
swiss_xmin = 2485071.58
swiss_ymin = 1074261.72
swiss_xmax = 2837119.8
swiss_ymax = 1299941.79
swiss_cell_size = 1_000

In [5]:
def create_grid(xmin, ymin, xmax, ymax, cell_size):
    width = xmax - xmin
    height = ymax - ymin
    rows = int(height // cell_size) + 1
    cols = int(width // cell_size) + 1
    
    grid = []
    for i in range(cols):
        for j in range(rows):
            x0 = xmin + i * cell_size
            y0 = ymin + j * cell_size
            x1 = x0 + cell_size
            y1 = y0 + cell_size
            grid.append(Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)]))
            
    return grid

Ne pouvant pas faire une grille uniquement sur la surface de la Suisse, la grille créée ci-dessus couvre donc également une partie des pays limitrophes. Une fois combinée avec les données collectée sur notre pays, il sera alors possible de n'utiliser que les cellules contenant des données et d'abandonner celles ne faisant pas partie du pays.

## Préparation des données
Dans cette section, nous allons préparer puis stocker les données qui seront utilisées, avant de les fusionner en un unique fichier plus tard.

Les fonctions ci-dessous permettre un traitement plus facile des données provenant de geo database.

In [6]:
def preproc_soil_aptitudes(row):
    # Changement de la valeur inconnue, pour une meilleure utilisation des catégories plus tard
    row['WASSERSPEI'] = ( 0.0 if float(str(row['WASSERSPEI'])) < 0.0 else row['WASSERSPEI'])
    row['NAHRSTOFF'] = ( 0.0 if float(str(row['NAHRSTOFF'])) < 0.0 else row['NAHRSTOFF'])
    row['WASSERDURC'] = ( 0.0 if float(str(row['WASSERDURC'])) < 0.0 else row['WASSERDURC'])
    
    return row

In [7]:
def preproc_habitat(row):
    row['TypoCH_NUM'] = int(str(row['TypoCH_NUM'])[:2])
    return row

In [8]:
def find_dominant_values(df, cell):
    
    # Vérifie si le DataFrame contient une seule ligne
    if len(df) == 1:
        return df
    
    # Calcule l'intersection entre la cellule et les geometry du df
    df['intersection_geometry'] = df['geometry'].apply(cell.intersection)
    
    # Récupère la surface des intersections
    df['intersection_area'] = df['intersection_geometry'].apply(lambda geom: geom.area if geom.is_valid else 0)
    
    # Si le DataFrame contient seulement deux lignes, retourne la ligne avec la plus grande intersection
    if len(df) == 2:
        dominant_row = df.loc[df['intersection_area'].idxmax()]
        return pd.DataFrame([dominant_row.drop(labels=['intersection_geometry', 'intersection_area'])])
    
    # Fonction pour trouver la valeur dominante pour chaque colonne
    def get_dominant_value(col):
        grouped = df.groupby(col)['intersection_area'].sum()
        return grouped.idxmax()
        
    # Trouver la valeur dominante pour chaque colonne
    dominant_values = {col: get_dominant_value(col) for col in df.columns if col not in ['geometry', 'intersection_geometry', 'intersection_area']}
    
    # Créer un DataFrame avec les valeurs dominantes
    dominant_df = pd.DataFrame([dominant_values])
    
    return dominant_df

In [9]:
def process_gdb(file_config, output_file, 
                xmin = swiss_xmin, ymin = swiss_ymin, xmax = swiss_xmax, ymax = swiss_ymax, cell_size=swiss_cell_size,
                max_df_size=10_000):
    
    # Créer une grille de carrés
    grid = create_grid(xmin, ymin, xmax, ymax, cell_size)
    
    # DataFrame pour accumuler les résultats
    result_df = gpd.GeoDataFrame()
    
    # Itérer sur chaque cellule de la grille
    for idx, cell in enumerate(grid):
        # Extraire les limites du rectangle pour le paramètre bbox
        bbox = cell.bounds
        
        paths = file_config['paths']
        layer_name = file_config['layer_name']
        columns = file_config['columns']
        
        # Lire seulement les données qui intersectent la cellule actuelle
        gdfs = []

        # Itérer sur chaque chemin dans 'paths'
        for path in paths:
            gdf = gpd.read_file(path, layer=layer_name, columns=columns, bbox=bbox)
            gdfs.append(gdf)

        if len(gdfs) > 0 :
            # Concaténer tous les lignes en en un seul
            cell_gdf = pd.concat(gdfs, ignore_index=True)
        
            if not cell_gdf.empty :
                cell_gdf = cell_gdf.apply(file_config['preproc_func'], axis=1)
                
                dominant_values = find_dominant_values(cell_gdf, cell)
                dominant_values['idx'] = idx
                dominant_values['geometry'] = cell

                dominant_values = dominant_values[['idx', 'geometry'] + [col for col in dominant_values.columns if col not in ['idx', 'geometry']]]
    
                if result_df.empty:
                    result_df = dominant_values
                else:
                    result_df = pd.concat([result_df, dominant_values], ignore_index=True)
                
                # Sauvegarder si le DataFrame devient trop grand
                if len(result_df) >= max_df_size:
                    if os.path.exists(output_file):
                        result_df.to_csv(output_file, mode='a', header=False, index=False)
                    else:
                        result_df.to_csv(output_file, index=False)
    
                    print(f"Enregistrement du dataframe actuel dans {output_file}")
                    result_df = result_df.head(0)
    
    # Sauvegarder les données restantes
    if not result_df.empty:
        if os.path.exists(output_file):
            result_df.to_csv(output_file, mode='a', header=False, index=False)
        else:
            result_df.to_csv(output_file, index=False)

    print(f"Enregistrement terminé dans {output_file}")


### Aptitudes des sols

Différentes propriétés du sol sont disponibles dans la carte des aptitudes du sol fournie par l’Office fédéral de l’agriculture. Les données, ainsi que la documentation officielle associée, sont accessibles au [téléchargement](https://data.geo.admin.ch/browser/index.html#/collections/ch.blw.bodeneignung-wasserdurchlaessigkeit/items/bodeneignung-wasserdurchlaessigkeit) . D'autres liens concernant cette carte, comme l'accès à l'API ou à l'aperçu de la carte sont disponibles sur ce [lien](https://www.geocat.ch/geonetwork/srv/fre/catalog.search#/metadata/d5bd22a3-0656-4582-97bc-37f77b5426ca).

Ces données contiennent plusieurs aptitudes différentes des sols, dont seulement certaines nous intéressent. Nous allons nous concentrer sur les caractéristiques suivantes :
- `Bodentyp` - Le type de sol
- `WASSERSPEI` - Capacité de rétention hydrique
- `NAHRSTOFF` - Capacité de rétention en substances nutritives 
- `WASSERDURC` - Perméabilité

Note : Les données étant volumineuses et non ajoutées sur github, il est important de vérifier que les données soient ajoutée au bon endroit et décompressées.

In [10]:
gdb_aptitude_path = "../../data/raw/Bodeneignungskarte_LV95.gdb"

In [11]:
aptitude_layers = list_available_layers(gdb_aptitude_path)

Couches disponibles : ['Bodeneignungskarte']


Pour les aptitudes du sol, une unique couche est disponible, nous n'avons donc pas besoin de les analyser pour savoir où se trouvent les données qui nous intéressent.

In [12]:
aptitude_columns = list_columns(gdb_aptitude_path, "Bodeneignungskarte")

Colonnes disponibles : ['AREA', 'PERIMETER', 'BEK200_', 'BEK200_ID', 'INT_CODE', 'CODE', 'Farbe', 'Shape_Leng', 'KU_CODE', 'Eignungsei', 'KULTURLAND', 'Bodentyp', 'GRUNDIGKEI', 'SKELETT', 'WASSERSPEI', 'NAHRSTOFF', 'WASSERDURC', 'VERNASS', 'Shape_Length', 'Shape_Area', 'geometry']


Comme annoncé plus tôt, le jeu de données contient de nombreuses caractéristiques, dont seulement une partie nous intéresse ici. Seules ces colonnes seront donc importées du geodatabase.

In [ ]:
# Définir les chemins et les paramètres
output_file_path  = "../../data/processed/processed_swiss_grid_soil_aptitude.csv"

aptitude_config = {
        'paths': ['../../data/raw/Bodeneignungskarte_LV95.gdb'],
        'layer_name': 'Bodeneignungskarte',
        'columns': ['Bodentyp', 'WASSERSPEI', 'NAHRSTOFF', 'WASSERDURC'],
        'preproc_func' : preproc_soil_aptitudes
    }

# Lancement du traitement
process_gdb(aptitude_config, output_file_path)

### Milieux naturels

La Suisse contient plusieurs milieux naturels différents, que l'Office Fédéral de l'environnement met à disposition sur GeoAdmin. Les données sont téléchargeables à ce [lien]().

GeoAdmin offre une [description visuelle](https://api3.geo.admin.ch/rest/services/api/MapServer/ch.bafu.lebensraumkarte-schweiz/legend?lang=fr) du jeu de données utilisé pour cette étape. Les données offrent une classification des différents habitats pour chaque zone géographique, avec jusqu'à 3 niveaux de précision. L'ensemble des milieux ne possédant pas 3 niveaux de précision, nous avons décidé de ne prendre en compte que les 2 premiers, afin de garder une séparation plus fine tout en conservant une information homogène.

In [13]:
gdb_environment_path = "../../data/raw/HabitatMap_CH_V1_20220805.gdb"

In [14]:
environment_layers = list_available_layers(gdb_environment_path)

Couches disponibles : ['HabitatMap_CH_V1_20220805', 'LUT', 'TypoCH_Catalogue', 'Probability_Catalogue']


Comme on peut le voir, plusieurs couches sont disponible. Les métadonnées indiquent quelques informations à leurs sujet :
- Couche `HabitatMap_CH_V1_20220805` : Couche principale contenant les données sur les milieux naturels
- Couche `LUT` : look-up table contenant des informations supplémentaires sur les milieux naturels, donc notament la source de la donnée
- Couche `TypoCH_Catalogue` : Catalogue des codes TypoCH afin de comprendre les valeurs présentes dans la couche principale
- Couche `Probability_Catalogue` : Catalogue afin de comprendre les valeurs de probabilité présentes dans la couche principale

Pour le moment, seule la couche `HabitatMap_CH_V1_20220805` nous est utile.

In [15]:
environment_columns = list_columns(gdb_environment_path, "HabitatMap_CH_V1_20220805")

Colonnes disponibles : ['TypoCH_NUM', 'Cover', 'POLYID', 'Version', 'Probability', 'TypoCH', 'Shape_Length', 'Shape_Area', 'geometry']


#### Séparation en plus petits fichiers
À cause de l'ampleur du jeu de données et de la complexité des géométries utilisées, il n'est pas possible de charger le jeu de données en une seule fois en mémoire. Afin de pouvoir faire des tests et une recherche plus rapide, il a été décider de séparer le jeu de données en plus petits fichiers, qui seront ensuite traités ensemble.

In [ ]:
# Définir le nombre d'entrées par fichiers
batch_size = 1_000_000

# Paramètres pour le fichier à séparer
gdb_path = '../../data/raw/HabitatMap_CH_V1_20220805.gdb'
layer_name = "HabitatMap_CH_V1_20220805"

# Emplacement de sauvegarde des fichier séparés
output_path = '../../data/raw/splited_HabitatMap'
output_filename = 'HabitatMap'

# Lecture du nombre total de lignes, en ne prenant qu'un seul attribut pour aller plus vite
total_rows = gpd.read_file(gdb_path, layer=layer_name, columns=['TypoCH_NUM'], ignore_geometry=True)

# Compteur pour les fichiers de sortie
file_counter = 0

# Lecture du fichier GDB par morceaux et enregistrement
with fiona.open(gdb_path, layer=layer_name) as source:
    file_counter += 1
    
    crs = source.crs
    schema = source.schema

    # Initialiser une liste pour stocker les features du chunk actuel
    features = []

    for i, feature in enumerate(source):
        features.append(feature)
        
        # Si on a atteint la taille du batch ou la fin des features
        if (i + 1) % batch_size == 0 or (i + 1) == total_rows:

            # Créer un GeoDataFrame à partir des features
            gdf_chunk = gpd.GeoDataFrame.from_features(features, crs=crs)
            
            # Définir le chemin de sortie pour ce lot
            output_gpkg = os.path.join(output_path, f"{output_filename}_chunk{file_counter}.gpkg")
            
            # Sauvegarder le lot dans un fichier GeoPackage
            gdf_chunk.to_file(output_gpkg, layer='HabitatMap_chunk',driver="GPKG")

            # Affichage de l'avancement
            print(f'Enregistrement de {i} lignes dans le fichier {output_gpkg}') 
            
            # Réinitialiser les features pour le prochain chunk
            features = []
            

print(f"Data split into {file_counter - 1} GDB files.")    

#### Préparation des données
Note : Malgré la séparation en plusieurs fichiers pour faciliter l'accès aux données, l'extraction des données de milieux naturel pour notre grille est relativement lent (compter entre 3 et 4 heures). Une optimisation du code serait sans doute possible, mais demanderai de nombreux tests de performances. Malheureusement, faute de temps, nous n'avons rien mis en place pour palier à cela.

In [ ]:
# Configuration avec génération dynamique des chemins
base_path = '../../data/raw/splited_HabitatMap'
file_prefix = 'HabitatMap_chunk'
file_extension = '.gpkg'

output_file_path  = "../../data/processed/processed_swiss_grid_environment.csv"

environment_config = {
    'paths': [
        os.path.join(base_path, f"{file_prefix}{i}{file_extension}")
        for i in range(1, file_counter + 1)
    ],
    'layer_name': 'HabitatMap_chunk',
    'columns': ['TypoCH_NUM'],
    'preproc_func' : preproc_habitat
}

process_gdb(environment_config, output_file_path)

### Altitude

Présente en partie dans nos jeu de données initial, l'altitude est importante afin de mieux modéliser l'environnement de la Suisse. Pour cela, nous avons choisis de prendre l'altitude moyenne pour chacun des pixels de notre grille.

Le modèle numérique du terrain MNT25 avec une maille de 200 m est un jeu de données qui décrit la surface terrestre en trois dimensions sans végétation ni constructions et est fournit par l'Office fédéral de topographie. Les données sont téléchargeables sur la [page officielle](https://www.swisstopo.admin.ch/fr/modele-altimetrique-mnt25-200m).    

Dans le code ci-dessous, nous avons utilisé la version `.xyz`, car ce format offrait la plus grande flexibilité et facilité d'utilisation à nos yeux.

Ce fichier utilisant des coordonnées LV03, nous devons les transformer en coordonnées LV95, avant de pouvoir calculer la moyenne d'altitude par cellule.

In [16]:
# Chargement du fichier XYZ
xyz_file = '../../data/raw/DHM25_200m/DHM200.xyz'
df_altitude = pd.read_csv(xyz_file, sep=r'\s+', names=['x', 'y', 'altitude'])

# Convertion les coordonnées de LV03 à LV95
transformer = pyproj.Transformer.from_crs("epsg:21781", "epsg:2056", always_xy=True)
df_altitude['x_lv95'], df_altitude['y_lv95'] = transformer.transform(df_altitude['x'].values, df_altitude['y'].values)

# Convertion en GeoDataFrame avec les coordonnées LV95
gdf_points = gpd.GeoDataFrame(df_altitude, geometry=gpd.points_from_xy(df_altitude.x_lv95, df_altitude.y_lv95), crs="EPSG:2056")

# Génération de la grille en utilisant la fonction create_grid
grid = create_grid(swiss_xmin, swiss_ymin, swiss_xmax, swiss_ymax, swiss_cell_size)
gdf_grid = gpd.GeoDataFrame({'geometry': grid}, crs="EPSG:2056")
gdf_grid.index.name = "idx"

# Spatial join des points avec les cellules
points_in_polygons = gpd.sjoin(gdf_points, gdf_grid, how='inner', predicate='within')

# Calcul de l'altitude moyenne pour chaque cellule et ajout dans le geodataframe
mean_altitude = points_in_polygons.groupby('idx').agg({'altitude': 'mean'})
gdf_altitude = gdf_grid.merge(mean_altitude, on='idx', how='left')

# Filtrer uniquement les polygones qui ont des valeurs calculées
gdf_altitude.dropna(subset=['altitude'], inplace=True)

gdf_altitude.head()

,geometry,altitude
idx,,
34,"POLYGON ((2485071.58 1108261.72, 2486071.58 11...",410.7500
35,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",363.5148
36,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",358.9348
37,"POLYGON ((2485071.58 1111261.72, 2486071.58 11...",387.6868
38,"POLYGON ((2485071.58 1112261.72, 2486071.58 11...",432.2572


In [17]:
# Enregistrement des données dans un fichier CSV
gdf_altitude.to_csv("../../data/processed/processed_swiss_grid_altitude.csv")

### Température moyenne annuelle
En nous basant sur Google Earth Engine, nous avons pu récupérer des informations concernant les températures moyennes annuelles sur tout le territoire suisse. Le script javascript utilisé pour obtenir les fichiers `.tif` utilisés se trouve dans le dossier `scripts`, sous le nom `terraclimate.js`.

In [18]:
mean_temp_data_riox = rioxarray.open_rasterio('../../data/raw/geo_tiffs/annual_mean_temperature_switzerland_bigger.tif')
mean_temp_data_riox.name = "data"
df = mean_temp_data_riox.squeeze().to_dataframe().reset_index()
geometry = gpd.points_from_xy(df.x, df.y)
mean_temp_data_gpd = gpd.GeoDataFrame(df, crs=mean_temp_data_riox.rio.crs, geometry=geometry)

# Suppression des entrées sans valeurs
mean_temp_data_gpd.dropna(inplace=True)

mean_temp_data_gpd.head()

C:\Users\Vicky\anaconda3\envs\GML\Lib\site-packages\pyproj\crs\_cf1x8.py:515: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


,y,x,band,spatial_ref,data,geometry
196,1299500.0,2681500.0,1,0,1.5,POINT (2681500 1299500)
197,1299500.0,2682500.0,1,0,1.5,POINT (2682500 1299500)
198,1299500.0,2683500.0,1,0,1.5,POINT (2683500 1299500)
199,1299500.0,2684500.0,1,0,1.5,POINT (2684500 1299500)
200,1299500.0,2685500.0,1,0,1.5,POINT (2685500 1299500)


Le geodataframe utilisant actuellement des points, nous allons modifier cela afin de travailler avec les cellules de la grille, comme pour les autres jeux de données. De cette manière, la fusion des jeux de données pourra se faire de manière plus homogène.

In [19]:
# Création de la grille
grid = create_grid(swiss_xmin, swiss_ymin, swiss_xmax, swiss_ymax, swiss_cell_size)

# Convertir la grille en GeoDataFrame
grid_gdf = gpd.GeoDataFrame({'geometry': grid}, crs=mean_temp_data_riox.rio.crs)

# Joindre les valeurs des deux dataframe
joined = gpd.sjoin(grid_gdf, mean_temp_data_gpd, how="left", predicate="contains")

# Supprimer les entrées sans valeur de température
joined.dropna(subset=['data'], inplace=True)

# Ne conserver que les colonnes utiles
joined = joined[['geometry', 'data']]

# Renommage des colonnes
joined.rename(columns={"data": "temperature"}, inplace=True)
joined.index.name = 'idx'

joined.head()

,geometry,temperature
idx,,
33,"POLYGON ((2485071.58 1107261.72, 2486071.58 11...",5.0
34,"POLYGON ((2485071.58 1108261.72, 2486071.58 11...",5.1
35,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",5.2
36,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",5.2
37,"POLYGON ((2485071.58 1111261.72, 2486071.58 11...",5.2


In [20]:
# Enregistrement des données dans un fichier CSV
joined.to_csv("../../data/processed/processed_swiss_grid_temperature.csv")

## Fusion des jeux de données
Maintenant que toutes nos données ont séparées sous forme de cellules de 1km de côté, nous pouvons les joindre en un unique jeu de données. Pour nous aider à cela plusieurs fonctions ont été crées.

Les données fusionnées ne sont cependant pas exemptes d'erreurs et une analyse exploratoire a été effectuées sur les entrées possédant des valeurs manquantes. L'ensemble des graphiques générés pour leur traitement, ainsi que quelques commentaires, sont disponibles dans le notebook associé, `eda_missing_swiss_data_merged`.

In [21]:
def load_geodataframes(file_paths):
    """Charge plusieurs fichiers CSV en GeoDataFrames et les stocke dans un dictionnaire."""
    geodfs = {}
    for name, file_path in file_paths.items():
        df = pd.read_csv(file_path)
        gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']))
        gdf.set_crs(epsg=2056, inplace=True)
        geodfs[name] = gdf
    return geodfs

In [22]:
def merge_geodataframes(geodfs):
    """Fusionne plusieurs GeoDataFrames en conservant toutes les entrées uniques en utilisant les noms relatifs pour les géométries."""
    # Initialiser avec le premier GeoDataFrame
    merged_gdf = geodfs.pop(next(iter(geodfs)))

   # Fusionner avec une jointure externe (outer join)
    for name, gdf in geodfs.items():
        merged_gdf = merged_gdf.merge(gdf, on='idx', how='outer', suffixes=('', f'_{name}_drop'))

    merged_gdf.set_index("idx", inplace=True)

    return merged_gdf

In [23]:
def resolve_geometry_columns(merged_gdf):
    """Résout les colonnes de géométrie supplémentaires en vérifiant et en consolidant les géométries."""
    # Trouver toutes les colonnes de géométrie
    geometry_columns = [col for col in merged_gdf.columns if col.startswith('geometry')]

    if len(geometry_columns) > 1:
        # Processus pour vérifier et consolider les géométries
        merged_gdf['geometry'] = merged_gdf[geometry_columns].apply(lambda row: next((geom for geom in row if not pd.isnull(geom)), None), axis=1)
        # Supprimer les colonnes de géométrie supplémentaires
        merged_gdf = merged_gdf.drop(columns=[col for col in geometry_columns if col != 'geometry'])

    return merged_gdf

Nous pouvons maintenant fusionner nos données en un unique geodataframe.

In [24]:
# Dictionnaire des chemins vers vos fichiers CSV avec un nom relatif
file_paths = {
    'alt' : '../../data/processed/processed_swiss_grid_altitude.csv', 
    'env' : '../../data/processed/processed_swiss_grid_environment.csv', 
    'apt' : '../../data/processed/processed_swiss_grid_soil_aptitude.csv',
    'tmp' : '../../data/processed/processed_swiss_grid_temperature.csv'
}

# Charger les GeoDataFrames
geodfs = load_geodataframes(file_paths)

# Fusionner les GeoDataFrames
merged_gdf = merge_geodataframes(geodfs)
merged_gdf.head()

,geometry,altitude,geometry_env_drop,TypoCH_NUM,geometry_apt_drop,Bodentyp,NAHRSTOFF,WASSERDURC,WASSERSPEI,geometry_tmp_drop,temperature
idx,,,,,,,,,,,
33,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,"POLYGON ((2485071.58 1107261.72, 2486071.58 11...",5.0
34,"POLYGON ((2485071.58 1108261.72, 2486071.58 11...",410.7500,None,NaN,None,NaN,NaN,NaN,NaN,"POLYGON ((2485071.58 1108261.72, 2486071.58 11...",5.1
35,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",363.5148,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",62.0,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,6.0,4.0,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",5.2
36,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",358.9348,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",62.0,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,6.0,4.0,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",5.2
37,"POLYGON ((2485071.58 1111261.72, 2486071.58 11...",387.6868,"POLYGON ((2485071.58 1111261.72, 2486071.58 11...",12.0,None,NaN,NaN,NaN,NaN,"POLYGON ((2485071.58 1111261.72, 2486071.58 11...",5.2


Les données dans ce nouveau Dataframe ne sont malheureusement pas obligatoirement sans incohérences. Les données provenant de plusieurs jeux différents, même si traitées sous la même forme, peuvent présenter de différences. C'est le cas, par exemple, pour l'altitude dont les données couvrent une zone beaucoup plus grande que juste la Suisse.

Il est donc intéressant de vérifier que lse différentes géometries sont les mêmes, dans le cas où il y en as plusieurs, puisque les données ont été jointes sur un autre attribut (`idx`). Ces vérifications, comme annoncé plus haut, ont été effectuée dans un notebook séparé.

In [25]:
def are_geometries_not_identical(row, geometry_columns):
    geometries = [row[col] for col in geometry_columns]
    
    # Filtrer les géométries non nulles
    geometries = [geom for geom in geometries if geom is not None]
    
    # Vérifier si toutes les géométries non nulles ne sont pas égales
    return not all(geom.equals(geometries[0]) for geom in geometries)

In [26]:
geometry_columns = [col for col in merged_gdf.columns if col.startswith('geometry')]
non_identical_gdf = merged_gdf[merged_gdf.apply(are_geometries_not_identical, geometry_columns=geometry_columns, axis=1)]
non_identical_gdf.shape[0]

0

Cette vérification ne nous retourne aucune ligne, ce qui signifie qu'aucune erreur de fusion n'a été faite.

Nous pouvons donc maintenant nous concentrer sur les différentes entrées pour lesquelles l'une ou l'autre des données n'existe pas. Pour cela, nous pouvons déjà afficher les informations des lignes ayant au moins une valeur nulle :

In [27]:
merged_gdf[merged_gdf.isnull().T.any()].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 15282 entries, 33 to 79683
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   geometry           14782 non-null  geometry
 1   altitude           14782 non-null  float64 
 2   geometry_env_drop  39 non-null     geometry
 3   TypoCH_NUM         39 non-null     float64 
 4   geometry_apt_drop  13 non-null     geometry
 5   Bodentyp           13 non-null     object  
 6   NAHRSTOFF          13 non-null     float64 
 7   WASSERDURC         13 non-null     float64 
 8   WASSERSPEI         13 non-null     float64 
 9   geometry_tmp_drop  6894 non-null   geometry
 10  temperature        6894 non-null   float64 
dtypes: float64(6), geometry(4), object(1)
memory usage: 1.4+ MB


Après analyse, les problèmes suivants ont été observés :
- Entrées avec uniquement l'altitude et/ou la température, dû à des cellules présentes en dehors des frontières suisses.
- Entrées sans altitude, dû à la présences de voies de communication présentes en Italie.
- Entrées sans milieu naturel, dû à une définition des frontières légèrement différente dans les jeux de données et à une intersection minime avec nos cellules.
- Entrées sans aptitudes du sol, dû à nouveau à une définition des frontières légèrement différente dans les jeux de données et à une intersection minime avec nos cellules.

Nous allons commencer par supprimer les entrées pour lesquels il a été déterminer de l'existence n'est pas pertinente dans notre analyse.

In [28]:
# Suppression des cellules hors de la Suisse
condition0 = (
    merged_gdf['geometry_env_drop'].isnull() &
    merged_gdf['geometry_apt_drop'].isnull()
)
merged_gdf = merged_gdf.loc[~condition0]

In [29]:
# Suppression des cellules hors de la Suisse (Voies de communication en Italie)
condition1 = (
    merged_gdf['geometry'].isnull() & 
    merged_gdf['geometry_apt_drop'].isnull() &
    (merged_gdf['TypoCH_NUM'] == 93)
)
merged_gdf = merged_gdf.loc[~condition1]

In [30]:
# Suppression des cellules sans milieu naturel
condition2 = (
   merged_gdf['geometry_env_drop'].isnull()
)

merged_gdf = merged_gdf.loc[~condition2]

In [31]:
# Suppression des cellules sans aptitudes du sol et ne devant pas être conservées
condition3 = (
    merged_gdf['geometry_apt_drop'].isnull() &
     (
         (merged_gdf['TypoCH_NUM']  == 12.0) | 
         (merged_gdf['TypoCH_NUM']  == 31.0) |
         (merged_gdf['TypoCH_NUM']  == 62.0) |
         (merged_gdf['TypoCH_NUM']  == 63.0) |
         (merged_gdf['TypoCH_NUM']  == 92.0) |
         (merged_gdf['TypoCH_NUM']  == 93.0)
     )
)
merged_gdf = merged_gdf.loc[~condition3]

Les cellules restantes peuvent toutes être conservées, mais doivent être marquées comme ayant des valeurs inconnues.

In [32]:
merged_gdf['Bodentyp'] = merged_gdf['Bodentyp'].fillna('-')
merged_gdf['NAHRSTOFF'] = merged_gdf['NAHRSTOFF'].fillna(0)
merged_gdf['WASSERDURC'] = merged_gdf['WASSERDURC'].fillna(0)
merged_gdf['WASSERSPEI'] = merged_gdf['WASSERSPEI'].fillna(0)

Maintenant que les problèmes de nos données ont été corrigées, nous pouvons nous débarasser des colonnes inutiles.

In [33]:
# Résoudre les colonnes de géométrie supplémentaires
resolved_gdf = resolve_geometry_columns(merged_gdf)

Nous faisons une dernière vérification, afin de valider qu'il ne reste plus aucunes entrées contenant des valeurs manquantes:

In [34]:
resolved_gdf[resolved_gdf.isnull().T.any()]

,geometry,altitude,TypoCH_NUM,Bodentyp,NAHRSTOFF,WASSERDURC,WASSERSPEI,temperature
idx,,,,,,,,


## Séparation des types de sols

La caractéristiques `Bodentyp` regroupe différents types de sols, avec une distinction de 2ème niveau. Les différentes cellules peuvent comporter plusieurs types de sols. Afin de simplifier l'utilisation de cette caractéristique, il a été décidé de séparer cette caractéristique en plusieurs caractéristiques différentes, une par type de sol. De cette manière, le type de sol pourra mieux être exploité dans nos différents tests.

In [ ]:
# TODO

## Enregistrement des données
Notre dataset étant maintenant bon à être utiliser, nous pouvont l'enregistrer pour y ajouter les données du néophyte. Cela sera fait dans un autre notebook.

In [35]:
# Sauvegarde du DataFrame dans un fichier CSV
filename = '../../data/processed/processed_enhanced_swiss_data.csv'
resolved_gdf.to_csv(filename, index=True, encoding='utf-8')

print(f"Le DataFrame a été sauvegardé dans le fichier '{filename}'")

Le DataFrame a été sauvegardé dans le fichier '../../data/processed/processed_enhanced_swiss_data.csv'
